# Daily Back Order Tracker (sub inventory added)

### **Data to update daily**:
1. Kevin's Back order report (Email from Kevin)
2. Part Original Inventory Data (Z-folder)
3. Parts RA Qty (Email from Sam)

### **Data to update regularly**:
1. GERP Master (Z-folder)
2. Sub Master Inventory (Z-folder)
3. EDW_Demand_6M (Z-folder)
4. Supplier Master Data (Z-folder)

In [87]:
!pip install msoffcrypto-tool -q

In [88]:
!pip install pyxlsb -q

In [89]:
# Importing Libraries

import pandas as pd
import numpy as np
import gcsfs
from datetime import datetime
from datetime import date, timedelta
import pyxlsb
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_columns', None)

In [90]:
folder_path = 'gs://scm_dataset/Daily BO data/'
file_name = '20241107 - Backorder and Hold Report.xlsx'

file_path = f'{folder_path}{file_name}'

fs = gcsfs.GCSFileSystem()
df_kevin_awaiting = pd.read_excel(fs.open(file_path), sheet_name='Awaiting Shipping Details', header=1, usecols="B:AR")

In [91]:
df_kevin_awaiting.head(1)

,WH,Order Status Code,Order No*,SO Line No*,Delivery No,Request No,Repair Receipt No,Order Date,Days Since Order Date,Awaiting Shipping Reason,Picking Release Date,Picking Release Timestamp,Aging Groups,Shipping Compliance Date,Shipping Compliance,Air/Ground,Shipping Method,Category,Hold Type,Ship Set Name,SO Type,Item Status,Parts No,Item Description,Parts Class Code,Key Parts Category,Part Functionality,Customer Type with FOC,Ship To Code,Ship To Customer Type,Ship To Name,Bill To Code,Bill To Full Name,Company Code,Division Name,Division Code,Sealed System Part Type,Master State,COGO Amount (USD),Order Amount (USD),Order Count,Order Qty,Order/COGO Amount (Awaiting Shipping)
0,ESC,AWAITING_SHIPPING,713456460,1.1,NaN,PON240405518939,NaN,2024-04-05,216,Back Order Hold,NaT,NaT,8. 60+ days,2024-04-08,Late,Ground,TRUCK-TL-TL Standard,Ground (Other),BACK_ORDER_HOLD,NaN,Regular_OMV_AI,Obsolete,ACQ30473820,"Cover Assembly,Rear",ACQ0633,N,FUNCTIONAL,ASC,23601200-S,Repair SVC Center,"A1 T.V. SALES AND SERVICE, L.L.C.",US067644001B,A1 TV SALES &,HE,LTV,GLT,NaN,MD,0.0,120.41,1,1,120.41


In [92]:
# Getting active items which are held as back order

df_kevin_awaiting2 = df_kevin_awaiting[[
    'Order Date', 'Days Since Order Date',
    'Awaiting Shipping Reason', 'Picking Release Date', 'Item Status',
    'Parts No', 'Parts Class Code','Item Description', 'Key Parts Category', 'Part Functionality',
    'Customer Type with FOC', 'Company Code', 'Division Code', 'Order Qty'
]].copy()

df_kevin_awaiting3 = df_kevin_awaiting2[(df_kevin_awaiting2['Awaiting Shipping Reason'] == 'Back Order Hold') & (df_kevin_awaiting2['Picking Release Date'].isna()) & (df_kevin_awaiting2['Item Status'] == 'Active')]

# deletion on unnecessary columns
df_kevin_awaiting3 = df_kevin_awaiting3.drop(columns=['Awaiting Shipping Reason', 'Picking Release Date', 'Item Status'])

# column name change
df_kevin_awaiting3 = df_kevin_awaiting3.rename(columns={
    'Days Since Order Date': 'Delay',
    'Item Description': 'Desc',
    'Part Functionality': 'Functionality',
    'Customer Type with FOC': 'Customer',
    'Key Parts Category': 'Key Parts',
    'Order Qty': 'BO qty'
})

In [93]:
df_kevin_awaiting3.head(2)

,Order Date,Delay,Parts No,Parts Class Code,Desc,Key Parts,Functionality,Customer,Company Code,Division Code,BO qty
1,2024-07-01,129,4959AR3402T,ACS0000,Curtain Assembly,N,FUNCTIONAL,PD Stock,HA,DGT,2
2,2024-07-05,125,EAJ65658001,EAJ0202,"LCD,Module-TFT",N,FUNCTIONAL,ASC,BS,GTT,2


In [94]:
df_kevin_awaiting3.shape

(2187, 11)

In [95]:
df_kevin_awaiting3['Parts No'].nunique()

899

In [96]:
df_kevin_awaiting3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2187 entries, 1 to 11054
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Order Date        2187 non-null   datetime64[ns]
 1   Delay             2187 non-null   int64         
 2   Parts No          2187 non-null   object        
 3   Parts Class Code  2187 non-null   object        
 4   Desc              2187 non-null   object        
 5   Key Parts         2187 non-null   object        
 6   Functionality     2187 non-null   object        
 7   Customer          2187 non-null   object        
 8   Company Code      2187 non-null   object        
 9   Division Code     2187 non-null   object        
 10  BO qty            2187 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(8)
memory usage: 205.0+ KB


In [97]:
folder_path = 'gs://scm_dataset/Daily BO data/'
file_name = '20241107 - Backorder and Hold Report.xlsx'

file_path = f'{folder_path}{file_name}'

# Read the Excel file with the second row as the header and skip the first two rows of data
summary = pd.read_excel(fs.open(file_path), sheet_name='Summary PN', header=1, usecols="B:BK", skiprows=[2])

# Select the necessary columns
summary = summary[['Parts No', 'On Hand Qty', 'In Staging Qty', 'WH Qty', 'Transit Qty', 'ETA']]
summary['ETA'] = summary['ETA'].replace('No ETA', 'N/A')
summary = summary.rename(columns={'On Hand Qty': 'On Hand', 'WH Qty': 'Warehouse', 'In Staging Qty': 'In Stage', 'Transit Qty': 'Transit'})

In [98]:
summary.head(1)

,Parts No,On Hand,In Stage,Warehouse,Transit,ETA
0,0CZZW1H004B,1,0,0,81,11/12_76 ; 12/02_5


In [99]:
summary.shape

(4294, 6)

In [100]:
# This dataframe shows the backordered parts with its inventory status

backorder_only = df_kevin_awaiting3.merge(summary, on='Parts No', how='left')
backorder_only.head(2)

,Order Date,Delay,Parts No,Parts Class Code,Desc,Key Parts,Functionality,Customer,Company Code,Division Code,BO qty,On Hand,In Stage,Warehouse,Transit,ETA
0,2024-07-01,129,4959AR3402T,ACS0000,Curtain Assembly,N,FUNCTIONAL,PD Stock,HA,DGT,2,0,0,0,0,N/A
1,2024-07-05,125,EAJ65658001,EAJ0202,"LCD,Module-TFT",N,FUNCTIONAL,ASC,BS,GTT,2,0,0,0,0,N/A


In [101]:
backorder_only.shape

(2187, 16)

In [102]:
# Part delivery Data

folder_path = 'gs://scm_dataset/Temp_INV/Original_Part_Inv/'
file_list = fs.glob(f'{folder_path}*.xlsx')

if file_list:
    file_path = file_list[0]
    purchasing_order = pd.read_excel(fs.open(file_path), header=1, usecols="A:P")
    purchasing_order = purchasing_order[['Original Part', 'open', 'OP AIR', 'OP TRK', 'OP SEA']].rename(columns={'Original Part': 'Parts No',
                                                                                                    'open': 'OPEN', 'OP AIR': 'AIR',
                                                                                                    'OP TRK': 'TRK', 'OP SEA': 'SEA'})
else:
    print("No files found in the specified folder.")

In [103]:
purchasing_order.head(1)

,Parts No,OPEN,AIR,TRK,SEA
0,AGF80300705,108000,0,0,108000


In [104]:
bo_parts_with_po = pd.merge(backorder_only, purchasing_order, how='left', on='Parts No')

In [105]:
bo_parts_with_po.head(1)

,Order Date,Delay,Parts No,Parts Class Code,Desc,Key Parts,Functionality,Customer,Company Code,Division Code,BO qty,On Hand,In Stage,Warehouse,Transit,ETA,OPEN,AIR,TRK,SEA
0,2024-07-01,129,4959AR3402T,ACS0000,Curtain Assembly,N,FUNCTIONAL,PD Stock,HA,DGT,2,0,0,0,0,N/A,0,0,0,0


In [106]:
# GERP master Data (detailed information on each Part)

folder_path = 'gs://scm_dataset/GERP_Master/new_gerp_master/'
file_list = fs.glob(f'{folder_path}*.xlsx')

if file_list:
    file_path = file_list[0]
    detail_info = pd.read_excel(fs.open(file_path), sheet_name='Data', header=1, usecols="A:AM")
    detail_info = detail_info[['Part', 'sales_model', 'first_receipt_date', 'parts_grade']].rename(columns={'Part': 'Parts No',
                                                                                                       'sales_model': 'Model', 'parts_grade': 'grade'})
else:
    print("No files found in the specified folder.")

In [107]:
info_added = pd.merge(bo_parts_with_po, detail_info, how='left', on='Parts No')

In [108]:
info_added.head(1)

,Order Date,Delay,Parts No,Parts Class Code,Desc,Key Parts,Functionality,Customer,Company Code,Division Code,BO qty,On Hand,In Stage,Warehouse,Transit,ETA,OPEN,AIR,TRK,SEA,Model,first_receipt_date,grade
0,2024-07-01,129,4959AR3402T,ACS0000,Curtain Assembly,N,FUNCTIONAL,PD Stock,HA,DGT,2,0,0,0,0,N/A,0,0,0,0,CP10G10.AWYAFRD,2021.09.02,C


In [109]:
# Accesorries Data

folder_path = 'gs://scm_dataset/ACC/ACC_LIST/'
file_list = fs.glob(f'{folder_path}*.xlsx')

if file_list:
    file_path = file_list[0]
    df_acc = pd.read_excel(fs.open(file_path), header=1, usecols="A:C")
    df_acc = df_acc.rename(columns ={'Part#': 'Parts No'})
    df_acc = df_acc.drop(columns='Desc.')
else:
    print("No files found in the specified folder.")

In [110]:
df_acc.head(2)

,Parts No,Acc.
0,5231JA2006A,Y
1,5231JA2006E,Y


In [111]:
# BER list data (Updated sometimes)

df_ber = pd.read_csv('gs://scm_dataset/BER/BER_LIST/LTV BER list.csv', encoding='utf-8-sig')
df_ber = df_ber.rename(columns ={'Part No': 'Parts No'})

In [112]:
df_ber.head(2)

,Parts No,BER Type
0,COV37112701,TV BER
1,COV37113001,TV BER


In [113]:
acc_added = pd.merge(info_added, df_acc, how='left', on='Parts No')

In [114]:
ber_added = pd.merge(acc_added, df_ber, how='left', on='Parts No')

In [115]:
ber_added.head(1)

,Order Date,Delay,Parts No,Parts Class Code,Desc,Key Parts,Functionality,Customer,Company Code,Division Code,BO qty,On Hand,In Stage,Warehouse,Transit,ETA,OPEN,AIR,TRK,SEA,Model,first_receipt_date,grade,Acc.,BER Type
0,2024-07-01,129,4959AR3402T,ACS0000,Curtain Assembly,N,FUNCTIONAL,PD Stock,HA,DGT,2,0,0,0,0,N/A,0,0,0,0,CP10G10.AWYAFRD,2021.09.02,C,NaN,NaN


In [116]:
# This cell does remove all the rows either if it is BER parts or Accessories

non_acc_ber = ber_added[(ber_added['Acc.'].isna()) & (ber_added['BER Type'].isna())]
non_acc_ber = non_acc_ber.drop(columns=['Acc.', 'BER Type'])

In [117]:
non_acc_ber.shape

(2045, 23)

In [118]:
non_acc_ber.head(1)

,Order Date,Delay,Parts No,Parts Class Code,Desc,Key Parts,Functionality,Customer,Company Code,Division Code,BO qty,On Hand,In Stage,Warehouse,Transit,ETA,OPEN,AIR,TRK,SEA,Model,first_receipt_date,grade
0,2024-07-01,129,4959AR3402T,ACS0000,Curtain Assembly,N,FUNCTIONAL,PD Stock,HA,DGT,2,0,0,0,0,N/A,0,0,0,0,CP10G10.AWYAFRD,2021.09.02,C


In [119]:
folder_path = 'gs://scm_dataset/Daily BO data/'
file_name = '5. Sub Master Inventory.xlsx'

file_path = f'{folder_path}{file_name}'

fs = gcsfs.GCSFileSystem()
sub_inv = pd.read_excel(fs.open(file_path), sheet_name='Sub Master Inventory', header=1, usecols="A:D")

In [120]:
sub_inv = sub_inv[sub_inv['sub inventory'] > 0]
sub_inv['sub inventory'] = sub_inv['sub inventory'].astype(int)

In [121]:
sub_inv.head(1)

,original_part,description,sub_part,sub inventory
0,ADQ74793501,"Filter Assembly,Water",AGF80300705,162007


In [122]:
sub_inv.shape

(30672, 4)

In [123]:
summed_sub_inv = sub_inv.groupby('original_part', as_index=False)['sub inventory'].sum()
summed_sub_inv.rename(columns={'sub inventory': 'total_sub_inventory'}, inplace=True)

sub_merged = non_acc_ber.merge(summed_sub_inv, left_on='Parts No', right_on='original_part', how='left')
sub_merged.drop(columns='original_part', inplace=True)
sub_merged['total_sub_inventory'] = sub_merged['total_sub_inventory'].fillna(0).astype(int)

In [124]:
sub_merged.shape

(2045, 24)

In [125]:
sub_merged.head(1)

,Order Date,Delay,Parts No,Parts Class Code,Desc,Key Parts,Functionality,Customer,Company Code,Division Code,BO qty,On Hand,In Stage,Warehouse,Transit,ETA,OPEN,AIR,TRK,SEA,Model,first_receipt_date,grade,total_sub_inventory
0,2024-07-01,129,4959AR3402T,ACS0000,Curtain Assembly,N,FUNCTIONAL,PD Stock,HA,DGT,2,0,0,0,0,N/A,0,0,0,0,CP10G10.AWYAFRD,2021.09.02,C,0


In [126]:
# EDW Data (Demand for each months per parts)

new_column_names = ['Part No', 'Desc', 'Division']

folder_path = 'gs://scm_dataset/Temp_INV/EDW_Demand_6M/'
file_list = fs.glob(f'{folder_path}*.xlsx')

if file_list:
    file_path = file_list[0]

    edw_demand = pd.read_excel(fs.open(file_path), header=None, skiprows=1)
    header_row = pd.read_excel(fs.open(file_path), header=None, nrows=1).iloc[0]
    edw_demand.columns = new_column_names + header_row[3:].tolist()
    edw_demand = edw_demand.iloc[:, :9]
else:
    print("No files found in the specified folder.")

In [127]:
edw_demand.head(1)

,Part No,Desc,Division,202405,202406,202407,202408,202409,202410
0,AGF80300704,"Package Assembly,C/SKD",REF,38028.0,89554.0,72892.0,46157.0,71744.0,12147.0


In [128]:
demand_merged = pd.merge(sub_merged, edw_demand, left_on='Parts No', right_on='Part No', how='left')

In [129]:
demand_merged.shape

(2045, 33)

In [130]:
demand_merged.drop(columns=['Desc_y', 'Division'], inplace=True)

In [131]:
print(demand_merged.columns)

Index([         'Order Date',               'Delay',            'Parts No',
          'Parts Class Code',              'Desc_x',           'Key Parts',
             'Functionality',            'Customer',        'Company Code',
             'Division Code',              'BO qty',             'On Hand',
                  'In Stage',           'Warehouse',             'Transit',
                       'ETA',                'OPEN',                 'AIR',
                       'TRK',                 'SEA',               'Model',
        'first_receipt_date',               'grade', 'total_sub_inventory',
                   'Part No',                202405,                202406,
                      202407,                202408,                202409,
                      202410],
      dtype='object')


In [132]:
columns_to_convert = [202405, 202406, 202407, 202408, 202409, 202410]

demand_merged[columns_to_convert] = demand_merged[columns_to_convert].fillna(0).astype(int)
demand_merged.drop(columns='Part No', inplace=True)

In [133]:
demand_merged.head(1)

,Order Date,Delay,Parts No,Parts Class Code,Desc_x,Key Parts,Functionality,Customer,Company Code,Division Code,BO qty,On Hand,In Stage,Warehouse,Transit,ETA,OPEN,AIR,TRK,SEA,Model,first_receipt_date,grade,total_sub_inventory,202405,202406,202407,202408,202409,202410
0,2024-07-01,129,4959AR3402T,ACS0000,Curtain Assembly,N,FUNCTIONAL,PD Stock,HA,DGT,2,0,0,0,0,N/A,0,0,0,0,CP10G10.AWYAFRD,2021.09.02,C,0,0,1,3,0,0,0


In [134]:
demand_merged['first_receipt_date'] = pd.to_datetime(demand_merged['first_receipt_date'], errors='coerce')

In [135]:
demand_merged.shape

(2045, 30)

In [136]:
def categorize_date(date_str):
      given_date = date_str

      current_date = datetime.now()

      diff_in_days = (current_date - given_date).days
      diff_in_months = diff_in_days // 30
      diff_in_years = diff_in_days // 365

      # Categorize based on the difference
      if diff_in_years >= 2:
          return "Over 2 Years"
      elif diff_in_years >= 1:
          return "Over 1 Year"
      elif diff_in_months > 6:
          return "Over 6 Months"
      elif diff_in_months > 3:
          return "Over 3 Months"
      else:
          return "Within 3 Months"

In [137]:
check_part_date_list =[]
for i in range(demand_merged.shape[0]):
  check_part_date_list.append(categorize_date(demand_merged['first_receipt_date'][i]))

demand_merged.insert(0, 'check_part_receipt_date', check_part_date_list)

In [138]:
demand_merged.head(1)

,check_part_receipt_date,Order Date,Delay,Parts No,Parts Class Code,Desc_x,Key Parts,Functionality,Customer,Company Code,Division Code,BO qty,On Hand,In Stage,Warehouse,Transit,ETA,OPEN,AIR,TRK,SEA,Model,first_receipt_date,grade,total_sub_inventory,202405,202406,202407,202408,202409,202410
0,Over 2 Years,2024-07-01,129,4959AR3402T,ACS0000,Curtain Assembly,N,FUNCTIONAL,PD Stock,HA,DGT,2,0,0,0,0,N/A,0,0,0,0,CP10G10.AWYAFRD,2021-09-02,C,0,0,1,3,0,0,0


In [139]:
columns_to_convert = [202405, 202406, 202407, 202408, 202409, 202410]
demand_merged.rename(columns={col: str(col) for col in columns_to_convert}, inplace=True)

In [140]:
demand_merged.sample(5)

,check_part_receipt_date,Order Date,Delay,Parts No,Parts Class Code,Desc_x,Key Parts,Functionality,Customer,Company Code,Division Code,BO qty,On Hand,In Stage,Warehouse,Transit,ETA,OPEN,AIR,TRK,SEA,Model,first_receipt_date,grade,total_sub_inventory,202405,202406,202407,202408,202409,202410
121,Over 1 Year,2024-11-05,2,AGM75509911,AGM0941,"Cover,Parts Assembly,SVC",N,FUNCTIONAL,ASC,HA,CVT,1,0,1,0,0,N/A,100,40,0,60,WDEP9423F.BRSLLGA,2023-03-16,B,0,11,16,10,16,16,17
235,Over 1 Year,2024-09-05,63,EAC64806502,EAC0200,"Rechargeable Battery,Lithium Ion",N,FUNCTIONAL,PD Stock,BS,GTT,2,0,0,0,0,N/A,21,0,0,21,14HK701G-WP.AUB,2023-10-11,D,0,2,0,0,0,2,0
1315,Over 1 Year,2024-10-24,14,AGL30067130,AGL0235,"Panel Assembly,Drawer",N,FUNCTIONAL,FOC,HA,DFT,1,0,0,0,29,11/19_3 ; 11/21_1 ; 12/03_2 ; 13/06_23,10,10,0,0,WM6500HWA.ABWEVUS,2023-05-17,C,0,7,8,5,13,7,12
346,Within 3 Months,2024-10-08,30,EBR42005004,EBR0100,"PCB Assembly,Main",N,FUNCTIONAL,Regional,HA,CVT,1,0,0,4,2,11/14_2,32,0,32,0,LRGL5823S.FSTELGA,2024-10-24,N,0,0,0,0,0,3,7
723,Over 2 Years,2024-10-29,9,AGM73590314,AGM0941,"Controller,Parts Assembly,SVC",N,FUNCTIONAL,PD Stock,HA,CVT,3,0,0,0,4,11/13_2 ; 11/26_2,12,0,2,10,LRG3081ST.FSTELGA,2014-07-01,C,0,0,0,2,0,0,4


In [141]:
demand_merged['Order Date'] = pd.to_datetime(demand_merged['Order Date'])

demand_merged_unique = demand_merged.drop_duplicates(subset=['Parts No', 'Order Date'])

df_unique = demand_merged_unique.groupby('Parts No')['Order Date'].apply(lambda x: x.tolist()).reset_index()

def calculate_14_day_streak(order_dates):
    if not order_dates:
        return 0

    max_date = max(order_dates)
    streak_count = sum(1 for date in order_dates if (max_date - pd.Timedelta(days=14)) <= date <= max_date)
    return streak_count

df_unique['Streak'] = df_unique['Order Date'].apply(calculate_14_day_streak)

In [142]:
df_unique = df_unique.drop('Order Date', axis=1)
streak_added = demand_merged.merge(df_unique, how='left', on='Parts No')

In [143]:
streak_added.head(1)

,check_part_receipt_date,Order Date,Delay,Parts No,Parts Class Code,Desc_x,Key Parts,Functionality,Customer,Company Code,Division Code,BO qty,On Hand,In Stage,Warehouse,Transit,ETA,OPEN,AIR,TRK,SEA,Model,first_receipt_date,grade,total_sub_inventory,202405,202406,202407,202408,202409,202410,Streak
0,Over 2 Years,2024-07-01,129,4959AR3402T,ACS0000,Curtain Assembly,N,FUNCTIONAL,PD Stock,HA,DGT,2,0,0,0,0,N/A,0,0,0,0,CP10G10.AWYAFRD,2021-09-02,C,0,0,1,3,0,0,0,1


In [144]:
folder_path = 'gs://scm_dataset/Parts_RA/'
file_list = fs.glob(f'{folder_path}*.xls')

if file_list:
    file_path = file_list[0]
    df_RA = pd.read_excel(fs.open(file_path))

df_RA2 = df_RA[(df_RA['KPI Flag'] == 'KPI') & (df_RA['Return Reason Code'] == 'PA1')]

WARNING *** file size (2140885) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [145]:
df_RA2.head(1)

,Fulfillment Month-YYYYMM,RMA No*,Fulfillment Date-YYYYMMDD,Fulfillment Week,Category,KPI Flag,Received Subinventory,Create Employee ID,Create Employee Name,Approval Name,Div Code,Premium?,GBU Code,ProdL1 Name,ProdL2 Name,ProdL3 Name,Model,Serial No (Request),Return Amount (USD),PAC(USD),Unit Selling Price,Compare Price,DR Type Code,Request No,Order Type,Order Line Type,Order Status Name,Approval ID,Approval Name.1,Agreement-Approval,Purchase Date-YYYYMMDD,Entry Date,Order Date,Agreement Req Date,Agreement Date,Approval Req Date,Approval Req Time,Approval Date-YYYYMMDD,*Hold Date,*Release Date,Delivery YYYYMMDD,Pickup YYYYMMDD,Line Close Date-YYYYMMDD,Cancel Date-YYYYMMDD,*Agreement Dept,Return Reason Code,Return Reason Name,Defect Code Name,Defect Code Desc,Defect Parts No,Defect Parts Class Desc,DealerName,⁬Bill To Customer Code,⁬Bill To Customer Eng Full Name,⁬Ship To Customer Code,Ship To ⁬Customer Eng Full Name,⁬Inventory Org Code,⁬Inventory Org Name,Subinventory Code,*Consignee Name,Consignee Name,Consignee Addr1 Info,Consignee Addr2 Info,Consignee City Name,Consignee State Name,Consignee Postal Code,Consignee Phone No,Consignee Mobile Phone No,Email,Remarks1,Remarks2,Special Code,NQA?,Department Code,Department Name,Input User Id,Input User Name,Customer Debit No (Line),Location Type,Service Receipt No,Invoice Number
4,202401,918306329,20240105,1,Buyback,KPI,UNITDMG-SV,YAI117040,DeLaCruz Joane,Sales System User,BS,Non-Premium,PCT,PC,Notebook PC,Notebook,17Z90Q-K,207NZLL047448,1049.99,868.584352,1049.99,NaN,C,DRN231228479562,DR_End_OMV_US,DR_End_Rec_Credit_OMV_US_L,NaN,AI100043,"Brooks, Kimberly . Brooks, Kimberly",Approved - Closed,03/31/2023,12/28/2023,12/28/2023,12/28/2023,12/28/2023,12/28/2023,11:11,12/28/2023,NaN,NaN,NaN,NaN,01/05/2024,<NULL>,Parts,PA1,PARTS NOT AVAILABLE_DELAYED SUPPLY,NaN,NaN,EBR38183202,PCB Assembly,COSTCO-S,US064908001B,LGEAI SERVICE NT,US0649080001S,LGEAI SERVICE NT,N8Z,HD Tech Warehouse (Defective Return),UNITDMG-SV,=????,NaN,NaN,NaN,CEDARBURG,WI,53012,4142487927,4142487927,JOANE.DELACRUZ@LGE.COM,RR/40444200/RNN231122094797/EBR38183202/CPS1/$...,FBB : $1049.99 PREAP_LM_SILS_12.19.2023,SILS,NaN,38945201,GCSC Training Account,PDELEON,Phoebe Love De Leon,NaN,NaN,NaN,NaN


In [146]:
# Count occurrences of each unique value in 'Defect Parts No'
part_counts = df_RA2['Defect Parts No'].value_counts()

# Convert the result to a DataFrame with custom column names
part_counts_df = part_counts.reset_index()
part_counts_df.columns = ['Parts No', 'RA_Counts']

In [147]:
RA_added = streak_added.merge(part_counts_df, how='left', on='Parts No')
RA_added['RA_Counts'] = RA_added['RA_Counts'].fillna(0).astype(int)

In [148]:
RA_added.head(1)

,check_part_receipt_date,Order Date,Delay,Parts No,Parts Class Code,Desc_x,Key Parts,Functionality,Customer,Company Code,Division Code,BO qty,On Hand,In Stage,Warehouse,Transit,ETA,OPEN,AIR,TRK,SEA,Model,first_receipt_date,grade,total_sub_inventory,202405,202406,202407,202408,202409,202410,Streak,RA_Counts
0,Over 2 Years,2024-07-01,129,4959AR3402T,ACS0000,Curtain Assembly,N,FUNCTIONAL,PD Stock,HA,DGT,2,0,0,0,0,N/A,0,0,0,0,CP10G10.AWYAFRD,2021-09-02,C,0,0,1,3,0,0,0,1,0


In [149]:
# Supplier Master Data

folder_path = 'gs://scm_dataset/Supplier/'
file_list = fs.glob(f'{folder_path}*.xlsx')

if file_list:
    file_path = file_list[0]
    df_supp = pd.read_excel(fs.open(file_path), sheet_name='Today', header=1, usecols="A:Z")
    df_supp2 = df_supp[['Part No', 'Supplier Code']].rename(columns ={'Part No': 'Parts No', 'Supplier Code': 'Supplier'})
else:
    print("No files found in the specified folder.")

In [150]:
supplier_added = pd.merge(RA_added, df_supp2, how='left', on='Parts No')
supplier_added.drop(columns='Desc_x', inplace=True)

In [151]:
supplier_added.head(1)

,check_part_receipt_date,Order Date,Delay,Parts No,Parts Class Code,Key Parts,Functionality,Customer,Company Code,Division Code,BO qty,On Hand,In Stage,Warehouse,Transit,ETA,OPEN,AIR,TRK,SEA,Model,first_receipt_date,grade,total_sub_inventory,202405,202406,202407,202408,202409,202410,Streak,RA_Counts,Supplier
0,Over 2 Years,2024-07-01,129,4959AR3402T,ACS0000,N,FUNCTIONAL,PD Stock,HA,DGT,2,0,0,0,0,N/A,0,0,0,0,CP10G10.AWYAFRD,2021-09-02,C,0,0,1,3,0,0,0,1,0,CN000081


In [152]:
category_df = pd.read_csv('gs://scm_dataset/category/part_category.csv', encoding='utf-8-sig')

In [153]:
category_df.head(1)

,AU+PCC,P/CATEGORY
0,CDZAAA0000,[DW] ACCESSORY ASS'Y


In [154]:
category_name_df = pd.read_csv('gs://scm_dataset/category/part_category_name.csv', encoding='utf-8-sig')

In [155]:
category_name_df.sample(1)

,Parts Class,Parts Class Desc
168,ACQ0683,"Cover Assembly,Sensor"


In [156]:
supplier_added['Division Code'].isna().sum()

0

In [157]:
def cat_desc(df, partno, part_catdf, class_catdf):
    pn = partno
    comp_code = df.loc[df['Parts No'] == pn, 'Division Code'].values[0]
    class_code = df.loc[df['Parts No'] == pn, 'Parts Class Code'].values[0]
    comp_code2 = comp_code[:2]+'Z'
    try:
        if class_code != None and comp_code != None:
            part_cat_code = part_catdf.loc[part_catdf['AU+PCC'] == comp_code2+class_code ,'P/CATEGORY'].values[0]
        else:
            part_cat_code = ""
    except IndexError:
        part_cat_code = ""
    try:
        if class_code != None and comp_code != None:
            class_name_code = class_catdf.loc[class_catdf['Parts Class'] == class_code, 'Parts Class Desc'].values[0]
        else:
            class_name_code = ""
    except IndexError:
        class_name_code = ""
    return  part_cat_code, class_name_code

In [158]:
# Lists to store the resulting data
part_category_list = []
description_name_list = []

# Looping through the DataFrame efficiently
for index, row in supplier_added.iterrows():
    part_category, class_desc = cat_desc(supplier_added, row['Parts No'], category_df, category_name_df)
    part_category_list.append(part_category)
    description_name_list.append(class_desc)

supplier_added['Category']= part_category_list
supplier_added['Desc'] = description_name_list

In [159]:
supplier_added[supplier_added['total_sub_inventory'] > 0].sample(5)

,check_part_receipt_date,Order Date,Delay,Parts No,Parts Class Code,Key Parts,Functionality,Customer,Company Code,Division Code,BO qty,On Hand,In Stage,Warehouse,Transit,ETA,OPEN,AIR,TRK,SEA,Model,first_receipt_date,grade,total_sub_inventory,202405,202406,202407,202408,202409,202410,Streak,RA_Counts,Supplier,Category,Desc
1246,Over 2 Years,2024-10-16,22,AGB74172403,AGB0194,N,FUNCTIONAL,FOC,HA,DVT,1,10,0,0,0,N/A,0,0,0,0,A907GMS.BIGELGA,2021-04-29,C,12,0,0,0,0,0,1,1,0,VN018311,[VCC] Nozzle,"Nozzle Assembly,Crevice"
357,Over 2 Years,2024-10-10,28,AAN75488611,AAN0000,N,FUNCTIONAL,PD Stock,HE,GLT,1,0,0,0,0,N/A,0,0,0,0,65UH615A.AUS,2016-05-06,D,4,0,0,0,0,0,1,1,0,KR044335,,Base Assembly
1477,Over 2 Years,2024-11-04,3,MEE64284901,MEE0628,N,FUNCTIONAL,DMS,HA,CVT,1,0,2,0,0,N/A,49,0,49,0,LREL6323S.FSTLLGA,2020-08-05,B,7,0,0,0,0,0,7,6,0,MX009896,[Cooking] Heater Ass'y,"Heater,Radiation"
1219,Over 2 Years,2024-10-11,27,AEB75304603,AEB0000,N,FUNCTIONAL,FOC,HA,CVT,1,0,0,0,26,11/18_14 ; 11/19_1 ; 11/25_11,0,0,0,0,SKSGR480GS.BSTESGN,2022-01-18,D,45,0,0,0,0,0,5,3,0,KR044335,[Cooking] Grille,Grille Assembly
1262,Over 2 Years,2024-10-16,22,EBR83845003,EBR0100,[REF] PCB,FUNCTIONAL,FOC,HA,CNT,1,0,0,0,0,N/A,0,0,0,0,LMXS28626D.ASBCNA0,2017-09-19,C,289,0,1,0,0,0,1,1,0,KR044335,[REF] PCB,PCB Assembly


In [166]:
def parse_eta(eta, order_date):
    if pd.isna(eta) or eta.strip() == '':  # Check if ETA is empty or NaN
        return []  # Return an empty list if there is no ETA data

    eta_entries = eta.split(';')
    eta_list = []
    for entry in eta_entries:
        if '_' in entry:
            date_str, qty_str = entry.split('_')
            date_str = date_str.strip()

            # Validate date format
            try:
                eta_date = datetime.strptime(date_str, "%m/%d")
                eta_date = eta_date.replace(year=order_date.year)
                eta_qty = int(qty_str)
                eta_list.append((eta_date.strftime("%m/%d/%Y"), eta_qty))
            except ValueError:
                print(f"Invalid date format in ETA: {date_str}")
                continue  # Skip invalid date entries

    return eta_list


In [167]:
supplier_added.sample(1)

,check_part_receipt_date,Order Date,Delay,Parts No,Parts Class Code,Key Parts,Functionality,Customer,Company Code,Division Code,BO qty,On Hand,In Stage,Warehouse,Transit,ETA,OPEN,AIR,TRK,SEA,Model,first_receipt_date,grade,total_sub_inventory,202405,202406,202407,202408,202409,202410,Streak,RA_Counts,Supplier,Category,Desc
368,Over 1 Year,2024-10-10,28,ADD73956046,ADD0322,N,CHECK CASE NOTES,PD Stock,HA,CNT,1,0,0,3,54,11/12_9 ; 11/13_9 ; 11/15_9 ; 11/18_18 ; 11/19...,153,0,153,0,LRFCC21D3S.ASTCNA0,2023-08-30,B,0,7,7,8,16,38,49,6,0,MX009896,[REF] Door,"Door Foam Assembly,Freezer"


In [168]:
def assign_flag(row):
    # This again goes to other function called 'parse_eta': ETA & Order Date columns ONLY
    eta_list = parse_eta(row['ETA'], row['Order Date'])
    on_hand_qty = row['On Hand']
    in_staging_qty = row['In Stage']
    wh_qty = row['Warehouse']
    order_qty = row['BO qty']
    order_date = row['Order Date']

    on_hand_quantities = on_hand_qty + in_staging_qty + wh_qty

    if not eta_list:
      if on_hand_quantities >= order_qty:
        return 'Ignore'
      else:
        return 'No ETA'

    # Check if the parts have ETA within 7 days and enough quantity
    sum_qty_7d = sum(qty for eta_date, qty in eta_list if datetime.strptime(eta_date, "%m/%d/%Y") <= order_date + timedelta(days=7))
    total_qty_7d = sum_qty_7d + on_hand_quantities

    if total_qty_7d >= order_qty:
        return 'ETA_in_1W'

    # Check if the parts have ETA within 14 days and enough quantity
    sum_qty_14d = sum(qty for eta_date, qty in eta_list if datetime.strptime(eta_date, "%m/%d/%Y") <= order_date + timedelta(days=14))
    total_qty_14d = sum_qty_14d + on_hand_quantities

    if total_qty_14d >= order_qty:
        return 'ETA_in_2W'

    return 'Flag'

In [169]:
supplier_added['Flag'] = supplier_added.apply(assign_flag, axis=1)

Invalid date format in ETA: 13/01
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/01
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/01
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/01
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/01
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/06
Invalid date format in ETA: 13/06
Invalid date f

In [170]:
supplier_added.head(3)

,check_part_receipt_date,Order Date,Delay,Parts No,Parts Class Code,Key Parts,Functionality,Customer,Company Code,Division Code,BO qty,On Hand,In Stage,Warehouse,Transit,ETA,OPEN,AIR,TRK,SEA,Model,first_receipt_date,grade,total_sub_inventory,202405,202406,202407,202408,202409,202410,Streak,RA_Counts,Supplier,Category,Desc,Flag
0,Over 2 Years,2024-07-01,129,4959AR3402T,ACS0000,N,FUNCTIONAL,PD Stock,HA,DGT,2,0,0,0,0,N/A,0,0,0,0,CP10G10.AWYAFRD,2021-09-02,C,0,0,1,3,0,0,0,1,0,CN000081,[RAC] Others,Curtain Assembly,No ETA
1,Over 2 Years,2024-08-19,80,EBZ37171602,EBZ0000,N,FUNCTIONAL,PD Stock,HA,CVT,1,0,0,0,10,11/15_10,40,0,40,0,LRG30355ST.*,2007-04-18,C,0,0,1,5,7,2,1,1,0,US016373,[Cooking] Others,Part,Flag
2,Over 2 Years,2024-09-10,58,EBR64458107,EBR0100,[W/M] PCB,FUNCTIONAL,ASC,HA,DFT,2,0,0,0,0,N/A,5,5,0,0,GCWP1069LS.ABWEELC,2014-11-19,C,0,0,0,0,0,4,0,1,0,KR044335,[WM] PCB,PCB Assembly,No ETA


In [172]:
supplier_added.rename(columns={
    'check_part_receipt_date': 'Part Age',
    'BO qty': 'BO',
    'On Hand': 'OH',
    'In Stage': 'IS',
    'Warehouse': 'WH',
    'Transit': 'Tran',
    'total_sub_inventory': 'SUB Inv',
    '202405': '05',
    '202406': '06',
    '202407': '07',
    '202408': '08',
    '202409': '09',
    '202410': '10',
    'RA_Counts': 'RA History'
}, inplace=True)

In [173]:
file_name = 'gs://scm_dataset/BO_file_result/final_output2.csv'
supplier_added.to_csv(file_name, index=False)